In [1]:
##importing the file of the combined transcripts and labels of alfano the students and raf etc.
import csv
import sys
import numpy as np
import pandas as pd
csv.field_size_limit(1000000)

combined = []
      
with open('combined3.csv', 'r',newline='', encoding='utf8') as f:
  csv_reader = csv.reader(f, delimiter=',')
  for i, row in enumerate(csv_reader):
    if i == 0:
      continue
    else:
      combined.append(row)

print(combined[0][1])

1


In [2]:
##splitting the data into test and training set
def dat_shuffle(combined):  
    X = []
    y = []                 
    for item in combined:
        if item[1] != 'x':
            X.append(item[0])
        if item[1] != 'x':
            if(int(item[1])==2 or int(item[1])==3):
               val = 0
            else:
               val = 1
            y.append(int(val))
    X = np.array(X)
    y = np.array(y)

    from sklearn.model_selection import StratifiedShuffleSplit
    indiced = StratifiedShuffleSplit(n_splits=20, test_size=0.2, random_state=4)

    for train_index, test_index in indiced.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
    training_data = []


    test_data = []

    for i, item in enumerate(X_train):
        training_data.append([str(X_train[i]), y_train[i]])
    for i, item in enumerate(X_test):
        test_data.append([str(X_test[i]), y_test[i]])
    


    return test_data, training_data, X_train, X_test,  y_train, y_test     

In [3]:
test_data, training_data, X_train, X_test,  y_train, y_test = dat_shuffle(combined)
test_data[1]

["[Applause] [Music]  what's up guys this is the honest out lot here and today we're gonna be doing a pretty funny video we're gonna be talking about seven guns that I wish I didn't buy now I'm gonna get some hate from the fanboys from this video and I understand why I'm not saying all these guns will be terrible for you I'm just saying for one reason or another they were terrible for me and I wish it didn't buy them in this list I'm gonna try to explain why I bought them originally why I hated them and some options that I would recommend buying instead so let's get started at number 7 my Wilson combat in 1911 now I got to admit I've wanted a wolfson combat for as long as I can remember all I've heard were awesome things about it the most reliable the most accurate 1911 on the planet with top-notch craftsmanship and a slide is smooth is a baby seals ass looks like a super model and it handles like a dream almost all of that is actually true believe it or not except for the reliability 

In [4]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import text, sequence
from keras.models import Sequential
from keras.layers import Embedding
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.models import Model, Input, Sequential
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, SpatialDropout1D, Activation
from keras.layers import Conv1D, Bidirectional, GlobalMaxPool1D, MaxPooling1D, BatchNormalization
from keras.optimizers import Adam

In [5]:
tokenizer = Tokenizer(num_words=20000)
tokenizer.fit_on_texts(list(X_train))

# turning the tokenized text into sequences
X_train = tokenizer.texts_to_sequences(X_train)
X_test  = tokenizer.texts_to_sequences(X_test)

# padding the sequences
X_train = sequence.pad_sequences(X_train,maxlen= 10000)
X_test  = sequence.pad_sequences(X_test,maxlen=10000)

print('X_train shape:', X_train.shape)
print('X_test shape: ', X_test.shape)

X_train shape: (385, 10000)
X_test shape:  (97, 10000)


In [6]:
X_train[1]

array([  0,   0,   0, ..., 318, 318, 137], dtype=int32)

In [7]:
# number of unique words we want to use (or: number of rows in incoming embedding vector)
max_features = 20000 

# max number of words in a comment to use (or: number of columns in incoming embedding vector)
max_len = 10000 

# dimension of the embedding variable (or: number of rows in output of embedding vector)
embedding_dims = 128

In [8]:
# instantiate NN model
base_model = Sequential()

# add embedding layer 
base_model.add(Embedding(input_dim=max_features, input_length=max_len,
                         output_dim=embedding_dims))

# add pooling layer 
# ... which will extract features from the embeddings of all words in the comment
base_model.add(GlobalMaxPool1D())

# add dense layer to produce an output dimension of 50 and apply relu activation
base_model.add(Dense(50, activation='relu'))

# set the regularizing dropout layer to drop out 30% of the nodes
base_model.add(Dropout(0.3))

# finally add a dense layer
# ... which projects output into six units and squash it with sigmoid activation
base_model.add(Dense(1, activation='sigmoid'))

In [9]:
base_model.compile(loss='binary_crossentropy',
                   optimizer="adam", metrics=['accuracy'])

# check the model with all our layers
base_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 10000, 128)        2560000   
_________________________________________________________________
global_max_pooling1d (Global (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 50)                6450      
_________________________________________________________________
dropout (Dropout)            (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 51        
Total params: 2,566,501
Trainable params: 2,566,501
Non-trainable params: 0
_________________________________________________________________


In [45]:
##Gives an error in anaconda but works in google colab 
base_hist = base_model.fit(X_train, y_train, batch_size=64, 
                             epochs=15, validation_split=0.1)

Epoch 1/15
6/6 [==============================] - 3s 537ms/step - loss: 0.6041 - accuracy: 0.7168 - val_loss: 0.5769 - val_accuracy: 0.7436
Epoch 2/15
6/6 [==============================] - 3s 492ms/step - loss: 0.5923 - accuracy: 0.7168 - val_loss: 0.5745 - val_accuracy: 0.7436
Epoch 3/15
6/6 [==============================] - 3s 488ms/step - loss: 0.5926 - accuracy: 0.7168 - val_loss: 0.5733 - val_accuracy: 0.7436
Epoch 4/15
6/6 [==============================] - 3s 486ms/step - loss: 0.5948 - accuracy: 0.7168 - val_loss: 0.5729 - val_accuracy: 0.7436
Epoch 5/15
6/6 [==============================] - 3s 483ms/step - loss: 0.5873 - accuracy: 0.7168 - val_loss: 0.5727 - val_accuracy: 0.7436
Epoch 6/15
6/6 [==============================] - 3s 490ms/step - loss: 0.5876 - accuracy: 0.7168 - val_loss: 0.5726 - val_accuracy: 0.7436
Epoch 7/15
6/6 [==============================] - 3s 489ms/step - loss: 0.5775 - accuracy: 0.7168 - val_loss: 0.5716 - val_accuracy: 0.7436
Epoch 8/15
6/6 [====

In [46]:
base_test_loss, base_test_auc = base_model.evaluate(X_test, y_test, batch_size=32)
print('Test Loss:    ', base_test_loss)
print('Test Accuracy:', base_test_auc)

4/4 [==============================] - 0s 50ms/step - loss: 0.5527 - accuracy: 0.7216
Test Loss:     0.5527227520942688
Test Accuracy: 0.7216494679450989


In [11]:
## importing  pretrained glove
MAX_SEQUENCE_LENGTH = 1000
MAX_NUM_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

# first, build index mapping words in the embeddings set
# to their embedding vector

print('Indexing word vectors.')

embeddings_index = {}
with open( 'glove.6B.100d.txt',encoding="utf8") as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, 'f', sep=' ')
        embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))

Indexing word vectors.
Found 347217 word vectors.


In [12]:
# constructing a matrix for the glove vectors
embedding_matrix = np.zeros((len(tokenizer.word_index)+1, 100))

for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [42]:
##implementing CNN with pretrained glove
glove_model = Sequential()

# add embedding layer 
glove_model.add(Embedding(input_dim =embedding_matrix.shape[0], input_length=10000,
                          output_dim=embedding_matrix.shape[1], 
                          weights=[embedding_matrix], trainable=False))
 
# set the dropout layer to drop out 50% of the nodes
glove_model.add(SpatialDropout1D(0.5))

# add convolutional layer that has ...
# ... 100 filters with a kernel size of 4 so that each convolution will consider a window of 4 word embeddings
glove_model.add(Conv1D(filters=100, kernel_size=4, padding='same', activation='relu'))
# add normalization layer
glove_model.add(BatchNormalization())
# add pooling layer 
glove_model.add(GlobalMaxPool1D())

# set the dropout layer to drop out 50% of the nodes
glove_model.add(Dropout(0.5))


# add dense layer to produce an output dimension of 50 and using relu activation
glove_model.add(Dense(50, activation='relu'))

# finally add a dense layer
glove_model.add(Dense(1, activation='sigmoid'))
glove_model.compile(loss='binary_crossentropy',
                    optimizer="adam",
                    metrics=['accuracy'])


glove_model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 10000, 100)        3524100   
_________________________________________________________________
spatial_dropout1d_5 (Spatial (None, 10000, 100)        0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 10000, 100)        40100     
_________________________________________________________________
batch_normalization_5 (Batch (None, 10000, 100)        400       
_________________________________________________________________
global_max_pooling1d_6 (Glob (None, 100)               0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 50)               

In [43]:
glove_hist = glove_model.fit(X_train, y_train, batch_size=64, 
                             epochs=15, validation_split=0.1)

Epoch 1/15
6/6 [==============================] - 19s 3s/step - loss: 5.2950 - accuracy: 0.4627 - val_loss: 0.9229 - val_accuracy: 0.2564
Epoch 2/15
6/6 [==============================] - 18s 3s/step - loss: 3.2799 - accuracy: 0.6678 - val_loss: 1.0325 - val_accuracy: 0.2564
Epoch 3/15
6/6 [==============================] - 18s 3s/step - loss: 2.0242 - accuracy: 0.6408 - val_loss: 0.9409 - val_accuracy: 0.2564
Epoch 4/15
6/6 [==============================] - 18s 3s/step - loss: 1.7468 - accuracy: 0.6099 - val_loss: 0.7536 - val_accuracy: 0.3333
Epoch 5/15
6/6 [==============================] - 18s 3s/step - loss: 1.8936 - accuracy: 0.6389 - val_loss: 0.7666 - val_accuracy: 0.2821
Epoch 6/15
6/6 [==============================] - 18s 3s/step - loss: 1.6965 - accuracy: 0.5851 - val_loss: 0.7543 - val_accuracy: 0.3333
Epoch 7/15
6/6 [==============================] - 18s 3s/step - loss: 1.3292 - accuracy: 0.6404 - val_loss: 0.6909 - val_accuracy: 0.5128
Epoch 8/15
6/6 [==================

In [44]:
## testing trained model on the test set
glove_test_loss, glove_test_auc = glove_model.evaluate(X_test, y_test, batch_size=64)
print('Test Loss:    ', glove_test_loss)
print('Test Accuracy:', glove_test_auc)

2/2 [==============================] - 1s 427ms/step - loss: 0.5898 - accuracy: 0.8454
Test Loss:     0.5898271799087524
Test Accuracy: 0.8453608155250549


In [16]:
##desinging a RNN
# instantiate pretrained glove model
glove_2_model = Sequential()

# add embedding layer 
glove_2_model.add(Embedding(input_dim =embedding_matrix.shape[0], input_length=10000,
                          output_dim=embedding_matrix.shape[1], 
                          weights=[embedding_matrix], trainable=False))

# set the dropout layer to drop out 50% of the nodes
glove_2_model.add(SpatialDropout1D(0.5))

# add bidirectional layer and pass in an LSTM()
glove_2_model.add(Bidirectional(LSTM(25, return_sequences=True)))

# add normalization layer
glove_2_model.add(BatchNormalization())

# add pooling layer 
glove_2_model.add(GlobalMaxPool1D())

# set the dropout layer to drop out 50% of the nodes
glove_2_model.add(Dropout(0.5))

# add dense layer to produce an output dimension of 50 and using relu activation
glove_2_model.add(Dense(50, activation='relu'))

# finally add a dense layer
glove_2_model.add(Dense(1, activation='sigmoid'))

In [21]:
glove_2_model.compile(loss='binary_crossentropy',
                    optimizer="adam",
                    metrics=['accuracy'])

glove_2_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 10000, 100)        3524100   
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 10000, 100)        0         
_________________________________________________________________
bidirectional (Bidirectional (None, 10000, 50)         25200     
_________________________________________________________________
batch_normalization_1 (Batch (None, 10000, 50)         200       
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 50)                0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 50)               

In [24]:
glove_2_hist = glove_2_model.fit(X_train, y_train, batch_size=64, 
                                 epochs=20, validation_split=0.1)

Epoch 1/20
6/6 [==============================] - 71s 11s/step - loss: 0.6230 - accuracy: 0.7044 - val_loss: 0.5563 - val_accuracy: 0.7692
Epoch 2/20
6/6 [==============================] - 68s 11s/step - loss: 0.5914 - accuracy: 0.7001 - val_loss: 0.5507 - val_accuracy: 0.7692
Epoch 3/20
6/6 [==============================] - 68s 11s/step - loss: 0.5850 - accuracy: 0.7338 - val_loss: 0.5462 - val_accuracy: 0.7692
Epoch 4/20
6/6 [==============================] - 68s 11s/step - loss: 0.5709 - accuracy: 0.7419 - val_loss: 0.5439 - val_accuracy: 0.7692
Epoch 5/20
6/6 [==============================] - 67s 11s/step - loss: 0.5752 - accuracy: 0.7278 - val_loss: 0.5427 - val_accuracy: 0.7692
Epoch 6/20
6/6 [==============================] - 70s 12s/step - loss: 0.5864 - accuracy: 0.7253 - val_loss: 0.5418 - val_accuracy: 0.7949
Epoch 7/20
6/6 [==============================] - 68s 11s/step - loss: 0.5927 - accuracy: 0.7204 - val_loss: 0.5384 - val_accuracy: 0.7949
Epoch 8/20
6/6 [===========

In [26]:
glove_2_test_loss, glove_2_test_auc = glove_2_model.evaluate(X_test, y_test, batch_size=32)
print('Test Loss:    ', glove_2_test_loss)
print('Test Accuracy:', glove_2_test_auc)

4/4 [==============================] - 4s 970ms/step - loss: 0.5383 - accuracy: 0.7938
Test Loss:     0.5382745265960693
Test Accuracy: 0.7938144207000732
